TODO

- Handle type mismatch and cast to proper type and not just to string
- metadata columns: (ingestion_timestamp, source_file_path, batch_id)
- [Optional] Dump corrupted or invalid rows (bad casts, missing columns) into quarantine bucket like error_records

In [ ]:
# spark.stop()

In [2]:
import os
import pyspark
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType
from google.cloud import storage

In [3]:
SPARK_MASTER = "spark://spark-master:7077"

spark = (
    SparkSession
    .builder
    # .master(SPARK_MASTER)
    .appName("process_type_mismatch")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar") # GCS Connector
    .getOrCreate()
)

# Google Cloud Service Account Credentials
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

spark

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 01:39:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
[
    ('patientid', 'string'),
    ('patientagegroup', 'string'),
    ('patientonsetage', 'integer'),
    ('patientonsetageunit', 'integer'),
    ('patientsex', 'integer'),
    ('patientweight', 'float'),
    ('serious', 'int'),
    ('seriousnessdeath', 'int'),
    ('seriousnesshospitalization', 'int'),
    ('seriousnessdisabling', 'int'),
    ('seriousnesslifethreatening', 'int'),
    ('seriousnessother', 'int'),
    ('receivedate', 'int'),
    ('receiptdate', 'int'),
    ('safetyreportid', 'int')
]

[('patientid', 'string'),
 ('patientagegroup', 'string'),
 ('patientonsetage', 'integer'),
 ('patientonsetageunit', 'integer'),
 ('patientsex', 'integer'),
 ('patientweight', 'float'),
 ('serious', 'int'),
 ('seriousnessdeath', 'int'),
 ('seriousnesshospitalization', 'int'),
 ('seriousnessdisabling', 'int'),
 ('seriousnesslifethreatening', 'int'),
 ('seriousnessother', 'int'),
 ('receivedate', 'int'),
 ('receiptdate', 'int'),
 ('safetyreportid', 'int')]

In [ ]:
def get_year(blob):
    return blob.name.split("/")[3]

def get_filename(blob):
    return blob.name.split('/')[-1].split('.')[0]

def scan_years(blobs): 
    return list({get_year(blob) for blob in blobs})

# def perform_cast(df):
#     df = (
#         df
#         .withColumn("patientid", F.col("patientid").cast(StringType()))
#         .withColumn("patientagegroup", F.col("patientagegroup").cast(StringType()))
#         .withColumn("patientonsetage", F.col("patientonsetage").cast(IntegerType()))
#         .withColumn("patientonsetageunit", F.col("patientonsetageunit").cast(IntegerType()))
#         .withColumn("patientsex", F.col("patientsex").cast(IntegerType()))
#         .withColumn("patientweight", F.col("patientweight").cast(FloatType()))
#         .withColumn("serious", F.col("serious").cast(IntegerType()))
#         .withColumn("seriousnessdeath", F.col("seriousnessdeath").cast(IntegerType()))
#         .withColumn("seriousnesshospitalization", F.col("seriousnesshospitalization").cast(IntegerType()))
#         .withColumn("seriousnessdisabling", F.col("seriousnessdisabling").cast(IntegerType()))
#         .withColumn("seriousnesslifethreatening", F.col("seriousnesslifethreatening").cast(IntegerType()))
#         .withColumn("seriousnessother", F.col("seriousnessother").cast(IntegerType()))
#         .withColumn("receivedate", F.col("receivedate").cast(IntegerType()))
#         .withColumn("receiptdate", F.col("receiptdate").cast(IntegerType()))
#         .withColumn("safetyreportid", F.col("safetyreportid").cast(IntegerType()))
#     )

#     return df

def process_parquet(bucket, dir, year):
    blobs = list(bucket.list_blobs(prefix=f"{dir}/{year}"))
    for blob in blobs:
        source_blob = f"gs://{bucket.name}/{blob.name}"
        destination_blob = f"gs://{bucket.name}/cleaned/pq/{dir.split('/')[-1]}/{year}/{get_filename(blob)}"

        print(f"Reading file {source_blob}")
        # Read parquet
        df = spark.read.parquet(source_blob)
        
        # Cast to right types

        # Transform

        # Write to Destination
        df.repartition(4).write.mode("overwrite").parquet(destination_blob)
        print(f"Saved to {destination_blob}")

In [ ]:
# client = storage.Client()
# bucket = client.get_bucket("zoomcamp-454219-ade-pipeline")
# dir = "data/pq/patient"
# year = '2004'

# test = '2004,2005,2006'

# if not test:
#     years = scan_years(list(test =bucket.list_blobs(prefix=dir)))
# else:
#     years = [s for s in test.split(',')]

# for year in years:
#     print(year)
#     # process_parquet(bucket, dir, year)

# # process_parquet(bucket,dir,year)

2004
2005
2006


In [ ]:
client = storage.Client()
bucket = client.get_bucket("zoomcamp-454219-ade-pipeline")
dirs = ["data/pq/patient", "data/pq/drug", "data/pq/reaction"]

# Iterate through each source directory and each year over n parquet files
for dir in dirs:
    # years = scan_years(bucket.list_blobs(prefix=dir))
    years = ['2004']
    for year in years:
        process_parquet(bucket, dir, year)

In [ ]:
class Patient:
    def __init__(self, df):
        self.df = df

    def cast(self):
        self.df = (
            self.df
            .withColumn("patientid", F.col("patientid").cast(StringType()))
            .withColumn("patientagegroup", F.col("patientagegroup").cast(StringType()))
            .withColumn("patientonsetage", F.col("patientonsetage").cast(IntegerType()))
            .withColumn("patientonsetageunit", F.col("patientonsetageunit").cast(IntegerType()))
            .withColumn("patientsex", F.col("patientsex").cast(IntegerType()))
            .withColumn("patientweight", F.col("patientweight").cast(FloatType()))
            .withColumn("serious", F.col("serious").cast(IntegerType()))
            .withColumn("seriousnessdeath", F.col("seriousnessdeath").cast(IntegerType()))
            .withColumn("seriousnesshospitalization", F.col("seriousnesshospitalization").cast(IntegerType()))
            .withColumn("seriousnessdisabling", F.col("seriousnessdisabling").cast(IntegerType()))
            .withColumn("seriousnesslifethreatening", F.col("seriousnesslifethreatening").cast(IntegerType()))
            .withColumn("seriousnessother", F.col("seriousnessother").cast(IntegerType()))
            .withColumn("receivedate", F.col("receivedate").cast(IntegerType()))
            .withColumn("receiptdate", F.col("receiptdate").cast(IntegerType()))
            .withColumn("safetyreportid", F.col("safetyreportid").cast(IntegerType()))
        )   

    def transform(self):
        self.df = self.df.withColumn(
            "patientagegroup",
            (
                F
                .when(F.col("patientagegroup") == '1', "Neonate")
                .when(F.col("patientagegroup") == '2', "Infant")
                .when(F.col("patientagegroup") == '3', "Child")
                .when(F.col("patientagegroup") == '4', "Adolescent")
                .when(F.col("patientagegroup") == '5', "Adult")
                .when(F.col("patientagegroup") == '6', "Elderly")
                .otherwise(None)
             )
        )

        # Normalize patientage
        self.df = self.df.withColumn(
            "patientage",
            (
                F
                .when(F.col("patientonsetageunit") == 800, F.col("patientonsetage") * 10)
                .when(F.col("patientonsetageunit") == 801, F.col("patientonsetage") * 1)
                .when(F.col("patientonsetageunit") == 802, F.col("patientonsetage") / 12)
                .when(F.col("patientonsetageunit") == 803, F.col("patientonsetage") / 52.143)
                .when(F.col("patientonsetageunit") == 804, F.col("patientonsetage") / 365.25)
                .when(F.col("patientonsetageunit") == 805, F.col("patientonsetage") / 8766)
                .otherwise(None)
            ).cast(FloatType())
        ).drop("patientonsetageunit", "patientonsetage")

        self.df = self.df.withColumn(
            "patientsex",
            (
                F
                .when(F.col("patientsex") == 1, "Male")
                .when(F.col("patientsex") == 2, "Female")
                .otherwise(None)
            ).cast(StringType())
        )

        self.df = self.df.withColumn(
            "patientweight",
            (
                F
                .when(
                    F.col("patientweight").rlike(r"^\d+(\.\d+)?$"),
                    F.col("patientweight").cast(FloatType()))
                .otherwise(None)
             )
        )

        self.df = self.df.withColumn(
            "serious",
            (
                F
                .when(F.col("serious") == 1, True)
                .when(F.col("serious") == 2, False)
                .otherwise(None)
             )
        )

        self.df = self.df.withColumn(
            "seriousnessdeath",
            (
                F
                .when(F.col("seriousnessdeath") == 1, True)
                .otherwise(False)
             )
        )

        self.df = self.df.withColumn(
            "seriousnesshospitalization",
            (
                F
                .when(F.col("seriousnesshospitalization") == 1, True)
                .otherwise(False)
             )
        )

        self.df = self.df.withColumn(
            "seriousnessdisabling",
            (
                F
                .when(F.col("seriousnessdisabling") == 1, True)
                .otherwise(False)
             )
        )

        self.df = self.df.withColumn(
            "seriousnesslifethreatening",
            (
                F
                .when(F.col("seriousnesslifethreatening") == 1, True)
                .otherwise(False)
             )
        )

        self.df = self.df.withColumn(
            "seriousnessother",
            (
                F
                .when(F.col("seriousnessother") == 1, True)
                .otherwise(False)
             )
        )